by wedad\
Hey team this file contains the optimisation for Linear Regression Optimzation and experimentations\
I'll be comparing them to the baseline model found in baseline_model.ipynb\
The columns dropped are selected from '4_FeatureEngineeringRegression' file

# LinearRegression Optimisation
Overall, there wasn't that many options when it came to optimizing Linear Regression model\
As there are no hyperparameters to tune, the approaches I took were feature selection, Different Linear Model testing\
And Ensemble of different models\
I also attempted to resample the data, similarly to the baseline model, but that attempt only showed minimal differences\
\
Overall the best performing model was the Ensemble with reduced feature set, as see below:
--------------------------------------------
Accuracy: Baseline:  0.835372 | Ensemble: 0.8446799836934364 \\ 
Precision: Baseline:  0.866667  |   Ensemble:   0.7931034482758621 \\
recall: Baseline:  0.075362   |  Ensemble:  0.11084337349397591 \\
f1_score: Baseline: 0.138667  |  Ensemble:  0.19450317124735728 \\
Although the precision Fell, the increase in recall and accuracy allowed for more well rounded model\\
Albeit it is a minor imporvent.


Read in all the relevant Libraries


In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
import warnings
warnings.filterwarnings('ignore')

In [26]:
seed_list = [1,2,3,4,5,6,7,8,9,10]

Read In the data

In [27]:
aki_df = pd.read_csv('../data/df_final_AKI.csv')
aki_df = aki_df.drop(columns=['Unnamed: 0', 'subject_id'])
aki_df

,dod,gender,age,Albumin,Creatinine,Hemoglobin,INR(PT),PT,Sodium,Urea Nitrogen,Arterial Blood Pressure diastolic,Arterial Blood Pressure systolic,Heart Rate,Respiratory Rate,hypertension,chronic_kidney_disease,sepsis,Intercept
0,0,1,73,3.6,1.2,9.4,3.2,13.8,138.0,16.0,48.0,101.0,84.0,20.0,1,0,0,1
1,0,0,24,4.1,0.5,10.2,1.2,15.7,143.0,6.0,56.0,114.0,59.0,16.0,0,0,0,1
2,0,0,54,4.1,0.4,10.2,1.0,14.4,142.0,22.0,48.0,130.0,59.0,24.0,1,0,0,1
3,0,1,84,3.5,1.1,10.2,1.6,18.7,144.0,17.0,56.0,114.0,89.0,21.0,1,0,0,1
4,0,0,59,3.6,0.6,10.2,1.7,12.4,153.0,8.0,70.0,91.0,99.0,21.0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9804,0,1,61,3.3,3.6,10.2,1.5,13.1,138.0,23.0,62.0,103.0,89.0,20.0,0,1,0,1
9805,0,1,74,3.5,0.6,10.2,1.2,13.6,138.0,6.0,56.0,114.0,107.0,22.0,1,0,0,1
9806,0,1,58,4.1,0.6,10.2,0.9,9.3,141.0,8.0,56.0,114.0,78.0,10.0,0,0,0,1
9807,1,1,84,4.0,3.4,12.2,1.1,17.9,141.0,88.0,56.0,114.0,101.0,25.0,1,1,0,1


In [28]:
model_desc = []
model_comparison_DF = pd.DataFrame(model_desc, columns=['model', 'accuracy', 'precision', 'recall', 'F1 score'])
model_comparison_DF

,model,accuracy,precision,recall,F1 score


The function below takes in different Linear Models, and tests them over number of different Seeds to get an Average perfro

In [29]:
x = aki_df.drop(columns=['dod'])
y = aki_df['dod']

def linearTester(model_desc, model, x, y, title='alg'):
    total_acc = 0
    total_pre = 0
    total_rec = 0
    total_f1 = 0
    total = len(seed_list)
    for rand in seed_list:
        x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.25, random_state=rand)
        model.fit(x_train, y_train)
        y_pred_lin = model.predict(x_test)
        y_pred_bin = [1 if y > 0.5 else 0 for y in y_pred_lin]
        total_acc += accuracy_score(y_test, y_pred_bin)
        total_pre += precision_score(y_test, y_pred_bin)
        total_rec += recall_score(y_test, y_pred_bin)
        total_f1 += f1_score(y_test, y_pred_bin)

    model_desc.append([
        str(model),
        total_acc/total,
        total_pre/total,
        total_rec/total,
        total_f1/total
        
                      ])
    # Evaluate
    print(f"{title}")
    print('--------------------------------------------')
    print("Accuracy:", total_acc/total,)
    print("Precision: ", total_pre/total,)
    print("recall: ", total_rec/total,)
    print("f1_score: ", total_f1/total,)
    print('--------------------------------------------')
    # print("Classification Report:\n", classification_report(y_test, y_pred_bin))
    plt.show()
    

In [30]:
lin_reg = linear_model.LinearRegression()
linearTester(model_desc, lin_reg, x, y,'Linear Regression baseline')

Linear Regression baseline
--------------------------------------------
Accuracy: 0.84027721157766
Precision:  0.7283479660626826
recall:  0.09012048192771084
f1_score:  0.16009707044429883
--------------------------------------------


Baseline model, accuracy of approximately 0.840\
It is important that due to the imbalance\ 
Predicting every value to be 0 gives 83% accuracy\
In our case, it is important that we focus on Precision and Recall

In [31]:
x = aki_df.drop(columns=['dod', 'Sodium','Hemoglobin','gender','hypertension','Intercept'])
y = aki_df['dod']
x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.25, random_state=seed_list[0])

In [32]:
linearTester(model_desc,lin_reg, x, y,'Linear Regression modified feature')

Linear Regression modified feature
--------------------------------------------
Accuracy: 0.8407256420709336
Precision:  0.746400832761646
recall:  0.08915662650602409
f1_score:  0.15905020006861326
--------------------------------------------


New Feature Baseline model, accuracy of approximately 0.841

In [33]:
alpha = [0,0.01,0.1,0.2,0.5,1,5,10,100]
solver = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
ridge_grid = {
    'alpha': alpha,
    'solver': solver,
}

In [34]:
ridge_grid = GridSearchCV(estimator = linear_model.Ridge(), param_grid = ridge_grid, cv = 3, verbose =2, n_jobs = 4)
ridge_grid.fit(x_train, y_train)
ridge_grid.best_params_

Fitting 3 folds for each of 63 candidates, totalling 189 fits


{'alpha': 1, 'solver': 'sparse_cg'}

In [35]:
ridge_model = linear_model.Ridge(alpha=0, solver='cholesky')
linearTester(model_desc, ridge_model, x, y,'Linear Ridge modified feature')

Linear Ridge modified feature
--------------------------------------------
Accuracy: 0.8407256420709336
Precision:  0.746400832761646
recall:  0.08915662650602409
f1_score:  0.15905020006861326
--------------------------------------------


In [36]:
alpha = [0.00001,0.001,0.01,0.1,0.2,0.5,1,5,10,50,100]
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
lasso_grid = {
    'alpha': alpha,
}
lasso_grid = GridSearchCV(estimator = linear_model.Lasso(), param_grid = lasso_grid, cv = 3, verbose =2, n_jobs = 4)
lasso_grid.fit(x_train, y_train)
lasso_grid.best_params_

Fitting 3 folds for each of 11 candidates, totalling 33 fits


{'alpha': 1e-05}

In [37]:
lasso_model = linear_model.Lasso(alpha=0.00001)
linearTester(model_desc, lasso_model, x, y,'Linear lasso modified feature')

Linear lasso modified feature
--------------------------------------------
Accuracy: 0.8406848756624543
Precision:  0.7455551667997009
recall:  0.0889156626506024
f1_score:  0.15864015735186618
--------------------------------------------


In [38]:
EN_model = linear_model.ElasticNet(alpha=0.1)
EN_model.fit(x_train, y_train)
linearTester(model_desc, EN_model, x, y,'Linear ElasticNet modified feature')

Linear ElasticNet modified feature
--------------------------------------------
Accuracy: 0.8333061557276805
Precision:  0.6832731532388283
recall:  0.029156626506024096
f1_score:  0.05580337837912709
--------------------------------------------


In [39]:
bayridge_model = linear_model.BayesianRidge()
bayridge_model.fit(x_train, y_train)
linearTester(model_desc, bayridge_model, x, y,'Linear Baysian Ridge modified feature')

Linear Baysian Ridge modified feature
--------------------------------------------
Accuracy: 0.8403179779861395
Precision:  0.7477754943305975
recall:  0.08506024096385542
f1_score:  0.15248960463494868
--------------------------------------------


## Ensembles
Create an Ensemble of all linear models to create a majority ruling model

In [40]:
def linearEnsemble(model_list, model_desc, x, y, title='Combined Ensemble'):
    prediction_list = []
    total_acc = 0
    total_pre = 0
    total_rec = 0
    total_f1 = 0
    total = len(seed_list)
    
    for rand in seed_list:
        prediction_list = []
        for model in model_list:
            x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.25, random_state=rand)
            model.fit(x_train, y_train)
            y_pred_lin = model.predict(x_test)
            y_pred_bin = [1 if y > 0.5 else 0 for y in y_pred_lin]
            prediction_list.append(y_pred_bin)

        prediction_list = [sum(x) for x in zip(*prediction_list)]
        final_prediction = []
        for pred in prediction_list:
            if pred >0:
                final_prediction.append(1)
            else:
                final_prediction.append(0)
                
        total_acc += accuracy_score(y_test, final_prediction)
        total_pre += precision_score(y_test, final_prediction)
        total_rec += recall_score(y_test, final_prediction)
        total_f1 += f1_score(y_test, final_prediction)


    model_desc.append([
        "Linear Regression Ensemble",
        total_acc/total,
        total_pre/total,
        total_rec/total,
        total_f1/total
                      ])
    
    # Evaluate
    print(f"{title}")
    print('--------------------------------------------')
    print("Accuracy:", total_acc/total)
    print("Precision: ", total_pre/total)
    print("recall: ", total_rec/total)
    print("f1_score: ", total_f1/total)
    print('--------------------------------------------')
    # print("Classification Report:\n", classification_report(y_test, y_pred_bin))
    plt.show()

In [41]:
model_list = [lin_reg, ridge_model, lasso_model, EN_model, bayridge_model]
linearEnsemble(model_list, model_desc, x, y, title='Ensemble Reduced Feature')

x = aki_df.drop(columns=['dod'])
y = aki_df['dod'] 
linearEnsemble(model_list, model_desc, x, y, title='Ensemble full Feature')

Ensemble Reduced Feature
--------------------------------------------
Accuracy: 0.8405625764370159
Precision:  0.7272573623175409
recall:  0.09301204819277108
f1_score:  0.1646699978545802
--------------------------------------------
Ensemble full Feature
--------------------------------------------
Accuracy: 0.8400733795352631
Precision:  0.7097678926444382
recall:  0.09349397590361447
f1_score:  0.1649257037541498
--------------------------------------------


In [42]:
model_comparison_DF = pd.DataFrame(model_desc, columns=['model', 'accuracy', 'precision', 'recall', 'F1 score'])
model_comparison_DF

,model,accuracy,precision,recall,F1 score
0,LinearRegression(),0.840277,0.728348,0.090120,0.160097
1,LinearRegression(),0.840726,0.746401,0.089157,0.159050
2,"Ridge(alpha=0, solver='cholesky')",0.840726,0.746401,0.089157,0.159050
3,Lasso(alpha=1e-05),0.840685,0.745555,0.088916,0.158640
4,ElasticNet(alpha=0.1),0.833306,0.683273,0.029157,0.055803
5,BayesianRidge(),0.840318,0.747775,0.085060,0.152490
6,Linear Regression Ensemble,0.840563,0.727257,0.093012,0.164670
7,Linear Regression Ensemble,0.840073,0.709768,0.093494,0.164926


It appears that the base linear Regression performs identical to Ridge and Lasso normalised iterations\
This is for iterations that utilised the reduced feature set. \
Creating an Ensemble of these models provides a small increase in accuracy and F1 scores

## Sampling Performance
We see from baseline that although the accuracy and precesion significantly decreases, the recall and f1 score improves significantly\
below is an attempt to incoperate the models into a ensemble framework

In [43]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.combine import SMOTEENN, SMOTETomek

In [44]:
x = aki_df.drop(columns=['dod', 'Sodium','Hemoglobin','gender','hypertension','Intercept'])
y = aki_df['dod']

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=0)
# Define models for each approach
models = [lin_reg, ridge_model, lasso_model, EN_model, bayridge_model]   
# Define sampling methods
sampling_methods = [None,  RandomOverSampler(),  SMOTE(), ADASYN(),RandomUnderSampler(), SMOTEENN(), SMOTETomek(), NearMiss()]

In [45]:
# Define a function to train and evaluate models
def resampled_eval(model, x_train, y_train, x_test, y_test, sampling_method, seed_list):
    
    total_acc = 0
    total_pre = 0
    total_rec = 0
    total_f1 = 0
    total = len(seed_list)
    
    for j in seed_list:
        if sampling_method:
            x_train_resampled, y_train_resampled = sampling_method.fit_resample(x_train, y_train)
        else:
            x_train_resampled = x_train
            y_train_resampled = y_train
            
        model.fit(x_train_resampled, y_train_resampled)
        y_pred = model.predict(x_test)
        y_pred = [1 if y > 0.5 else 0 for y in y_pred]

        # Evaluate
        total_acc += accuracy_score(y_test, y_pred)
        total_pre += precision_score(y_test, y_pred)
        total_rec += recall_score(y_test, y_pred)
        total_f1 += f1_score(y_test, y_pred)
        
    print(f"{str(sampling_method)}")
    print('--------------------------------------------')
    print("Accuracy:", total_acc/total,)
    print("Precision: ", total_pre/total,)
    print("recall: ", total_rec/total,)
    print("f1_score: ", total_f1/total,)
    print('--------------------------------------------')
    
    return

In [46]:
print("Sampling Methods")
for samp in sampling_methods:
    resampled_eval(lin_reg, x_train, y_train, x_test, y_test, samp, seed_list)

Sampling Methods
None
--------------------------------------------
Accuracy: 0.8440366972477065
Precision:  0.7708333333333333
recall:  0.11144578313253015
f1_score:  0.19473684210526312
--------------------------------------------
RandomOverSampler()
--------------------------------------------
Accuracy: 0.7176860346585117
Precision:  0.33064545156143244
recall:  0.6524096385542169
f1_score:  0.4388638035192371
--------------------------------------------
SMOTE()
--------------------------------------------
Accuracy: 0.7206422018348624
Precision:  0.32043851382455013
recall:  0.5807228915662652
f1_score:  0.41298798441587276
--------------------------------------------
ADASYN()
--------------------------------------------
Accuracy: 0.713914373088685
Precision:  0.3147591490018116
recall:  0.5867469879518072
f1_score:  0.40971999872921916
--------------------------------------------
RandomUnderSampler()
--------------------------------------------
Accuracy: 0.7173802242609582
Precision

## Sampling Ensemble
Attempt to create an Ensemble version of the sampled models

In [47]:
def linearEnsemble_Resampled(model_list, model_desc, sampling_methods, x, y,seed_list, majority_vote, title='Combined Resampled Ensemble'):
    final_prediction = []
    total_acc = 0
    total_pre = 0
    total_rec = 0
    total_f1 = 0
    total = len(seed_list)
    
    for rand in seed_list:
        prediction_list = []
        x_train, x_test, y_train, y_test = train_test_split(x, y,stratify=y, test_size=0.2, random_state=rand)
        for model in model_list: 
            for samp in sampling_methods: 
                if samp != None:
                    x_train, y_train = samp.fit_resample(x_train, y_train)
                model.fit(x_train, y_train)
                y_pred_lin = model.predict(x_test)
                y_pred_bin = [1 if y > 0.5 else 0 for y in y_pred_lin]
                prediction_list.append(y_pred_bin)
#                 prediction_list = [a + b for a, b in zip(prediction_list, y_pred_bin)]
# #                 prediction_list.append(y_pred_bin)
                
        prediction_list = [sum(x) for x in zip(*prediction_list)]
        final_prediction = []
        for pred in prediction_list:
            if pred > majority_vote:
                final_prediction.append(1)
            else:
                final_prediction.append(0)
                
        # print(set(prediction_list))
        total_acc += accuracy_score(y_test, final_prediction)
        total_pre += precision_score(y_test, final_prediction)
        total_rec += recall_score(y_test, final_prediction)
        total_f1 += f1_score(y_test, final_prediction)

    # Evaluate
    print(f"{title}")
    print('--------------------------------------------')
    print("Accuracy:", total_acc/total)
    print("Precision: ", total_pre/total)
    print("recall: ", total_rec/total)
    print("f1_score: ", total_f1/total)
    print('--------------------------------------------')
    # print("Classification Report:\n", classification_report(y_test, y_pred_bin))
    plt.show()
    
    return

In [48]:
model_desc = []
sampling_methods = [RandomOverSampler(),  SMOTE(), ADASYN(),RandomUnderSampler(), SMOTEENN(), SMOTETomek(), NearMiss()]
model_list = [lin_reg, ridge_model, lasso_model, EN_model, bayridge_model]
#35 total runs, majority rule 20>
# but for certainty and higher acc threshold of 30 is set
votes = [4, 9, 14,19, 24, 29, 33, 34]
for v in votes:
    linearEnsemble_Resampled(model_list, model_desc,sampling_methods, x, y, seed_list, v, title=f'Sampled Ensemble >{v+1}')

Sampled Ensemble >5
--------------------------------------------
Accuracy: 0.6418960244648317
Precision:  0.28521049139431875
recall:  0.7406626506024095
f1_score:  0.41178016025428443
--------------------------------------------
Sampled Ensemble >10
--------------------------------------------
Accuracy: 0.6902650356778797
Precision:  0.31133801410457285
recall:  0.6843373493975903
f1_score:  0.42787770191965785
--------------------------------------------
Sampled Ensemble >15
--------------------------------------------
Accuracy: 0.6919469928644242
Precision:  0.31224368869291613
recall:  0.6810240963855421
f1_score:  0.4280883564744709
--------------------------------------------
Sampled Ensemble >20
--------------------------------------------
Accuracy: 0.6968399592252803
Precision:  0.316187778913103
recall:  0.6795180722891565
f1_score:  0.4314691737179227
--------------------------------------------
Sampled Ensemble >25
--------------------------------------------
Accuracy: 0.703